In [8]:
import geowombat as gw
from geowombat.util import Cluster
import dask
import rasterio as rio
from rasterio.windows import Window
# from dask.distributed import Client, LocalCluster

In [3]:
from geowombat.data import rgbn, rgbn_suba, rgbn_subb

In [5]:
def dummy_func(block, dummy):
    return block * 10.0 / 2.0

In [73]:
cluster = Cluster(n_workers=2,
                  threads_per_worker=4,
                  scheduler_port=0,
                  processes=False)

cluster.start()

cluster.client

Client Scheduler: inproc://172.16.11.249/30495/136 Dashboard: http://localhost:8787/status,Cluster Workers: 4 Cores: 16 Memory: 16.54 GB


In [74]:
def n_rows_cols(pixel_index, block_size, rows_cols):
    return block_size if (pixel_index + block_size) < rows_cols else rows_cols - pixel_index

In [75]:
chunksize = 64

@dask.delayed
def write_func(output, out_window, out_indexes, last_write=None):
    
    del last_write
    
    with rio.open('/home/jcgr/Documents/scripts/Python/git_repos/geowombat/data/test2.tif',
                  mode='r+', height=dss.shape[0], width=dss.shape[1],
                  count=1, dtype='float64', driver='GTiff',
                  crs=ds.crs, transform=ds.transform, 
                  sharing=False) as dst:    
    
        dst.write(output, window=out_window, indexes=out_indexes)  

    
with gw.open(rgbn_suba, chunks=chunksize) as ds:
    
    dss = ds.sel(band=4).astype('float64')
    dss.attrs = ds.attrs
    
    with rio.open('/home/jcgr/Documents/scripts/Python/git_repos/geowombat/data/test2.tif',
                  mode='w', height=dss.shape[0], width=dss.shape[1],
                  count=1, dtype='float64', driver='GTiff',
                  crs=ds.crs, transform=ds.transform, 
                  sharing=False) as dst:
        pass

    writer = None
    for i in range(0, dss.shape[0], chunksize):
        nrows = n_rows_cols(i, chunksize, dss.shape[0])
        for j in range(0, dss.shape[1], chunksize):
            ncols = n_rows_cols(j, chunksize, dss.shape[1])
            w = Window(row_off=i, col_off=j, height=nrows, width=ncols)
            writer = write_func(dss[i:i+chunksize, j:j+chunksize].squeeze().data, 
                                w, 1, writer)

    res = cluster.client.persist(writer)

In [77]:
cluster.stop()

AttributeError: 'NoneType' object has no attribute 'close'

In [6]:
chunksize = 64

gw.apply(rgbn_suba,
         '/home/jcgr/Documents/scripts/Python/git_repos/geowombat/data/test.tif',
         dummy_func,
         args=(None,),
         n_jobs=4,
         count=4,
         blockxsize=chunksize,
         blockysize=chunksize)

# with gw.open(rgbn_suba, chunks=chunksize) as ds:
#     ds.gw.to_raster('/home/jcgr/Documents/scripts/Python/git_repos/geowombat/data/test.tif',
#                     n_jobs=4,
#                     tiled=True)

100%|██████████| 99/99 [00:00<00:00, 10311.30it/s]


In [ ]:
writer.compute()

In [ ]:
res = cluster.client.persist(writer)

In [ ]:
# res.compute()

In [40]:
cluster.stop()

In [ ]:
# writer.visualize()

In [ ]:
dss.data.visualize()

In [ ]:
with gw.open(rgbn_suba, band_names=['b', 'g', 'r', 'n']) as ds:
    ds.gw.imshow(band_names=['b', 'g', 'r'], flip=True, mask=True, nodata=0, robust=True)

In [ ]:
with gw.open(rgbn_subb, band_names=['b', 'g', 'r', 'n']) as ds:
    ds.gw.imshow(band_names=['b', 'g', 'r'], flip=True, mask=True, nodata=0, robust=True)

In [ ]:
with gw.open([rgbn_suba, rgbn_subb], band_names=['b', 'g', 'r', 'n'], mosaic=True) as ds:
    ds.gw.imshow(band_names=['b', 'g', 'r'], flip=True, mask=True, nodata=0, robust=True)

In [ ]:
with gw.open([rgbn_suba, rgbn_subb, rgbn_subb], 
             time_names=[1, 1, 2], 
             band_names=['b', 'g', 'r', 'n']) as ds:
    
    ds.sel(time=1).gw.imshow(band_names=['b', 'g', 'r'], flip=True, mask=True, nodata=0, robust=True)

In [ ]:
import rasterio
from rasterio.warp import calculate_default_transform, aligned_target
from rasterio.crs import CRS

In [ ]:
with rasterio.open(rgbn) as src:
    
    transform, width, height = calculate_default_transform(src.crs,
                                                           CRS.from_epsg(4326),
                                                           src.width,
                                                           src.height,
                                                           *src.bounds,
                                                           dst_width=src.width,
                                                           dst_height=src.height)
    
    print(width, height)
    res = (transform[0], -transform[4])
    
    print(res)
    
    dst_transform, dst_width, dst_height = aligned_target(transform, width, height, res)
    
    print(dst_transform)
    print(dst_width, dst_height)

In [ ]:
with gw.config.update(sensor='rgbn', ref_res=(5, 5)):

    with gw.open(rgbn, band_names=['b', 'g', 'r', 'n'], resampling='cubic') as ds:

        dss = ds.gw.moving(stat='max', n_jobs=8)

        dss.gw.imshow(band_names=['g', 'r', 'n'], flip=True, mask=True, nodata=0, robust=True)

In [ ]:
with gw.config.update(sensor='rgbn', ref_res=(5, 5)):

    with gw.open(rgbn, 
                 band_names=['b', 'g', 'r', 'n'],
                 resampling='cubic') as ds:
        
        print(ds)
        ds.gw.imshow(band_names=['b', 'g', 'r'], flip=True, mask=True, nodata=0, robust=True)